## Steps
1. Reproducing kaggle https://www.kaggle.com/code/jamesloy/deep-learning-based-recommender-systems
2. Recreate all models in neural collaborative filtering

In [1]:
import os

os.chdir("/Users/yenchenchou/Documents/GitHub/recsys-benchmarks")

In [2]:
import random
import polars as pl
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets

In [56]:
df = pl.read_csv(
    "data/2017-ncf-paper-data/ml-1m.train.rating",
    separator="\t",
    has_header=False,
    new_columns=["user_id", "movie_id", "rating", "timestamp"],
)
df.head()

user_id,movie_id,rating,timestamp
i64,i64,i64,i64
0,32,4,978824330
0,34,4,978824330
0,4,5,978824291
0,35,4,978824291
0,30,4,978824291


In [25]:
df.shape

(994169, 4)

In [49]:
df.group_by(["user_id"], maintain_order=True).head(n=1)

user_id,movie_id,rating,timestamp
i64,i64,i64,i64
0,32,4,978824330
1,86,4,978300174
2,179,4,978298486
3,210,4,978294282
4,18,2,978246576
5,400,4,978238948
6,430,4,978234874
7,468,5,978233526
8,30,4,978226678


In [53]:
df_test = pl.read_csv(
    "data/2017-ncf-paper-data/ml-1m.test.rating",
    separator="\t",
    has_header=False,
    new_columns=["user_id", "movie_id", "rating"],
    schema={"user_id": pl.Int32, "movie_id": pl.Int32, "rating": pl.Int32},
)
df_test.head()

ValueError: specified column names do not start with 'column_', but autogenerated header names were requested

In [27]:
df_test.shape

(994169, 3)

In [28]:
df_test_negative = pl.read_csv(
    "data/2017-ncf-paper-data/ml-1m.test.negative",
    separator="\t",
    has_header=False,
)
df_test_negative.head()

column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,column_37,…,column_64,column_65,column_66,column_67,column_68,column_69,column_70,column_71,column_72,column_73,column_74,column_75,column_76,column_77,column_78,column_79,column_80,column_81,column_82,column_83,column_84,column_85,column_86,column_87,column_88,column_89,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99,column_100
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""(0,25)""",1064,174,2791,3373,269,2678,1902,3641,1216,915,3672,2803,2344,986,3217,2824,2598,464,2340,1952,1855,1353,1547,3487,3293,1541,2414,2728,340,1421,1963,2545,972,487,3463,2727,…,1770,881,1913,1803,1734,3326,1617,224,3352,1869,1182,1331,336,2517,1721,3512,3656,273,1026,1991,2190,998,3386,3369,185,2822,864,2854,3067,58,2551,2333,2688,3703,1300,1924,3118
"""(1,133)""",1072,3154,3368,3644,549,1810,937,1514,1713,2186,660,2303,2416,670,1176,788,889,3120,2344,2525,3301,2055,1436,2630,11,2773,2176,1847,740,2332,3561,263,3658,3282,1980,2093,…,3540,3341,2973,1580,2118,3511,526,1719,525,1520,486,557,1353,500,2902,1687,1295,2997,2415,797,2518,926,3537,1746,1676,1875,3029,1535,341,3525,1429,2225,1628,2061,469,3056,2553
"""(2,207)""",2216,209,2347,3,1652,3397,383,2905,2284,2866,584,783,3208,1534,2529,1907,1170,3037,2015,1045,3099,3298,2522,739,2652,3702,792,2527,1945,2333,1668,3511,70,1991,3071,2474,…,2831,1548,1630,2227,2352,760,350,302,791,300,3528,1444,2,798,997,376,2565,1565,718,710,2695,904,3643,655,3666,3069,3661,953,865,813,1353,2945,2580,2989,2790,2879,2481
"""(3,208)""",3023,1489,1916,1706,1221,1191,2671,81,2483,941,841,1617,1437,2700,1904,1763,1181,599,2442,1656,1370,1171,1372,1444,1596,2023,1456,2179,530,1949,2628,290,1996,2593,2489,3484,…,2007,1937,2244,55,1016,2382,2506,3501,914,3127,23,3187,799,2572,1038,3028,2619,1429,2623,2158,2785,3674,2578,1837,1689,296,959,3347,1707,2901,2767,2167,1921,247,1618,2016,2323
"""(4,222)""",1794,3535,108,593,466,2048,854,1378,1301,697,1376,22,2135,3657,3173,1322,976,6,1399,817,2757,2010,652,458,1227,204,592,2875,1930,2251,1654,2542,1149,3089,454,2466,…,2524,932,3327,187,2575,1674,557,2547,1572,776,1600,2682,2085,1987,1390,614,3098,1831,927,2285,1059,2850,3517,134,1852,2776,1694,2490,1332,2526,2804,2027,833,176,463,2851,2453


In [29]:
df_test_negative.shape

(6040, 100)

In [45]:
from ast import literal_eval
neg_users, neg_items = [], []
with open("data/2017-ncf-paper-data/ml-1m.test.negative", "r") as f:
    line = f.readline()
    while line != None and line != "":
        arr = line.split("\t")
        user = literal_eval(arr[0])[0]
        negatives = [int(item) for item in arr[1:]]
        users = [user] * len(negatives)
        neg_users.extend(users)
        neg_items.extend(negatives)
        line = f.readline()
df_tmp = pl.DataFrame(
    {
        "user_id": neg_users,
        "movie_id": neg_items,
        "rating": [0] * len(neg_items),
    }
)

In [3]:
from torch import tensor
from torchmetrics.retrieval import RetrievalPrecision

In [7]:
indexes = tensor([0, 0, 0, 1, 1, 1, 1])
preds = tensor([0.2, 0.3, 0.5, 0.1, 0.3, 0.5, 0.2])
target = tensor([False, False, True, False, True, False, True])
p2 = RetrievalPrecision(top_k=2, aggregation="mean")
p2(preds, target, indexes=indexes)


tensor(0.5000)